# 1. Import and Install Dependencies

In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras import models
from keras import layers
from tensorflow.python.client import device_lib 


In [ ]:
# Check GPU
print(device_lib.list_local_devices())

# 2. Load Data

In [ ]:
dataset_path1 = r"AllData\Characters\Peter\Characters_Dataset"
dataset_path2 = r"AllData\Characters\Aya"

In [ ]:
classes = [ 'aleff',  'bb', 'ta', 'thaa', 'jeem', 'haa', 'khaa', 'dal', 'thal', 'ra', \
         'zay' , 'seen', 'sheen',  'saad', 'dhad','taa', 'dha', 'ain', 'ghain','fa', \
          'gaaf', 'kaaf','laam', 'meem', 'nun', 'ha', 'waw','ya']

### Prepare Data for VGG16

In [ ]:
dataset1 = image_dataset_from_directory(dataset_path1,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(224,224),color_mode='rgb',label_mode= 'categorical',
                                             class_names=classes)

In [ ]:
dataset2 = image_dataset_from_directory(dataset_path2,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(224,224),color_mode='rgb',label_mode= 'categorical',
                                             class_names=classes)

In [ ]:
# Combine 2 Datasets
dataset = dataset1.concatenate(dataset2)
# Get No. of Batches
tf.data.experimental.cardinality(dataset)

In [ ]:
# Split to Training and Validation
dataset_batches = tf.data.experimental.cardinality(dataset)
# 30% of Data To Validation and Testing
validation_dataset = dataset.take(dataset_batches // 7)
# 70% of Data To Training
training_dataset = dataset.skip(dataset_batches // 7) 


In [ ]:
# Split the Validation to valid and test
val_batches = tf.data.experimental.cardinality(validation_dataset)
testing_dataset = validation_dataset.take(val_batches // 2)
validation_dataset = validation_dataset.skip(val_batches // 2)


In [ ]:
# To Increase up Feeding Data To Network
AUTOTUNE = tf.data.AUTOTUNE
training_dataset = training_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
testing_dataset = testing_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Display The No. of Batches 
print(tf.data.experimental.cardinality(training_dataset))
print(tf.data.experimental.cardinality(validation_dataset))
print(tf.data.experimental.cardinality(testing_dataset))

# 3. Initialize Data Augmentation Layer

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
# Custom Layer for Brightness and Contrast
class RandomColorDistortion(tf.keras.layers.Layer):
    def __init__(self, contrast_range=[0.6, 1.8], 
                 brightness_delta=[-0.3, 0.3], **kwargs):
        super(RandomColorDistortion, self).__init__(**kwargs)
        self.contrast_range = contrast_range
        self.brightness_delta = brightness_delta
    
    def call(self, images, training=True):
        if not training:
            return images
        
        contrast = np.random.uniform(
            self.contrast_range[0], self.contrast_range[1])
        brightness = np.random.uniform(
            self.brightness_delta[0], self.brightness_delta[1])
        
        images = tf.image.adjust_contrast(images, contrast)
        images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 255)
        return images

In [ ]:
# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    # Make a Random Rotation
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.05),
    # Make a Random Width
    tf.keras.layers.RandomWidth((-.2,.2), interpolation='bilinear'),
    # Make a Random Height
    tf.keras.layers.RandomHeight((-.2,.2), interpolation='bilinear'),
    # Make a Random Translation
    tf.keras.layers.RandomTranslation((-0.1, 0.1),(-0.1, 0.1),fill_mode='constant'),
    # Resize image to 224 x 224 pixels again after Widthing,Heighting
    tf.keras.layers.Resizing(224,224,interpolation='bilinear'),
    # Make a Random Color
    RandomColorDistortion()
])

In [ ]:
# Show The Output of Augmentation Layer for Same Image
for image, _ in training_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))

    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224,224)

# 4. Model VGG16

In [ ]:
# Get The Base Network
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [ ]:
# Freeze Conv Layer
conv_base.trainable = False

##### Build a Network for Training

In [ ]:
# Build a Network for Training 
inp = layers.Input(shape=(224,224,3))
x = data_augmentation(inp)
x = (conv_base)(inp)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outs = layers.Dense(len(classes), activation='softmax')(x)
model = models.Model(inputs=inp, outputs=[outs])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

In [ ]:
log_dir = os.path.join('Logs/vgg16_transfer_learning')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/vgg16_transfer_learning.h5', monitor='val_acc', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1,patience=5)
callbacks = [tb_callback,mc,es]

In [ ]:
model.fit(training_dataset,epochs=50,callbacks=[callbacks], validation_data = validation_dataset)

In [ ]:
# Save The Last Model
model.save('Models/last_training_model.h5')

## 4.1 Model Evaluation

In [ ]:
model.evaluate(testing_dataset) 